<a href="https://colab.research.google.com/github/pythonnewb420/Personal_Projects/blob/main/Turning_Circle_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MIT License

Copyright (c) 2019 Wok

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


First step is to import necessary libraries before executing any code in this notebook

In [ ]:
import pandas as pd #means the pandas framework will now be useable by calling it "pd"
import numpy as np #means the pandas framework will now be useable by calling it "np"
import tensorflow as tf #means the pandas framework will now be useable by calling it "tf"
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

After Importing the necessary libraries, we can now import our data (please note that pandas accept .csv format files so that it can convert it into it's own matrix called dataframe, it is similar to an excel file). The code below is to [upload our data to Colab ](https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=JiJVCmu3dhFa)

In [ ]:
data_kapal = pd.read_csv("/content/Hasil_Perhitungan_TCircle_NILAI ONLY.csv",header=None, error_bad_lines=False)
data_kapal.drop(0,axis=1,inplace=True)
data_kapal.drop(0,axis=0,inplace=True)
display(data_kapal)

In [ ]:
data_kapal.describe()

In [ ]:
data_kapal.isnull().any()

In [ ]:
data_kapal.dropna(axis=1,how='any')

data_kapal.drop(index=data_kapal.index[0],axis=0,inplace=True)
data_kapal = data_kapal.apply(pd.to_numeric, errors='coerce')
print(data_kapal)

In [ ]:
y = np.array(data_kapal.pop(3),dtype='float32')
x = np.array(data_kapal,dtype='float32')
print(np.isnan(x.any()))
print(np.isnan(y.any()))
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,shuffle=True)

# Machine Learning Pipeline

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=100,
    decay_rate=0.6)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x_train)

In [ ]:
model = tf.keras.models.Sequential([
    normalizer,
    tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,name="output")
    ]
)
model.compile(loss=tf.keras.losses.MeanSquaredLogarithmicError(),
              optimizer=optimizer,
              metrics=['mae'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',
    patience=200,
    verbose=0,
    mode='auto',
)

In [ ]:
model.build(x.shape)
model.summary()

In [ ]:
%%time
history = model.fit(
    x_train,y_train,
    validation_split = 0.2,
    batch_size = 20,
    verbose=0, 
    epochs=3000,
    callbacks=[early_stopping])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 5])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Turnin_Circle]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
acc1, mse1 = model.evaluate(x_test, y_test)
print(f"Accuracy is {acc1*100} %,\nMSE is {mse1}")

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')

In [ ]:
model.save('/content/saved_model/Model_Riset')
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/Model_Riset

To save data into new GSheet

sh = gc.create('Predicted Hydrodynamics Coefficient')

# Open our new sheet and add some data.
worksheet = gc.open('Predicted Hydrodynamics Coefficient').sheet1
worksheet.update_cells(cell_list)
## Go to https://sheets.google.com to see your new spreadsheet.